# Connect Four 

## Introduction

Our Project is an AI that plays the game [ConnectFour](https://en.wikipedia.org/wiki/Connect_Four)

### Authors

- Oliver Gorges
- Fabian Stölzle

### Challenges

There are [4531985219092](https://oeis.org/A212693) possible states in the game ConnectFour with a 6x7 grid

    This Results in:
        - A very large set of possible States, which is nearly impossible to discover completely
        - A lot of time to train the AI til it discovered enough states to play against a real Player
        - A lot of storage space is needed to store the Action-Value Function
    


### How to Play

#### Train the AI

Before you can play the game against the AI you should train it at least for `1000000` episodes.

> To play the game without trainig you can also download a [JSON](drive.google.com/file/d/1aWwLZWc6jVqUtHLk88wz48rknHi-ohjI/view?usp=sharing) with trained Data.
> This will not work with Jupyter notebook

#### Start the Game

To play the game against the ai, you have to change the `play` parameter in the `Game Configuration` to True 

When you run now the project it will print the playing Field and you can start playing the game
```bash
    [['-' '-' '-' '-' '-' '-' '-']
     ['-' '-' '-' '-' '-' '-' '-']
     ['-' '-' '-' '-' '-' '-' '-']
     ['-' '-' '-' '-' '-' '-' '-']
     ['-' '-' '-' '-' '-' '-' '-']
     ['-' '-' '-' 'O' '-' '-' '-']]
    -------------------------------
    [['1' '2' '3' '4' '5' '6' '7']]
    Player 2, Enter Postion (1-7): 
```
> When you use oldData it can take a while to load the trained data and display the gamefield

> In the current state of the Project you start always as the secound player, so the ai has done its first turn already.

#### Game Rules

##### Inputs

The player can input a number between 1 and the configured amount of columns (`columns_count`, Default: 7). 
This will place a stone in the first empty row in the selected column.

##### Goal

To win the game you have to create a row out of four stones in a Horizontal, Vertical or Diagonal direction.

```bash
    Horizontal Win
    [['-' '-' '-' '-' '-' '-' '-']
     ['-' '-' '-' '-' '-' '-' '-']
     ['-' '-' '-' 'O' '-' '-' '-']
     ['-' '-' '-' 'O' '-' '-' '-']
     ['-' '-' 'X' 'O' '-' '-' '-']
     ['-' '-' 'X' 'O' '-' '-' 'X']]
    -------------------------------
    [['1' '2' '3' '4' '5' '6' '7']]
    Player 2, Enter Postion (1-7): 
```

```bash
    Vertical Win
    [['-' '-' '-' '-' '-' '-' '-']
     ['-' '-' '-' '-' '-' '-' '-']
     ['-' '-' '-' '-' '-' '-' '-']
     ['-' '-' '-' '-' '-' '-' '-']
     ['-' '-' '-' 'X' 'X' '-' '-']
     ['-' 'O' 'O' 'O' 'O' 'X' '-']]
    -------------------------------
    [['1' '2' '3' '4' '5' '6' '7']]
    Player 2, Enter Postion (1-7): 
```

```bash
    Diagonal Win
    [['-' '-' '-' '-' '-' '-' '-']
     ['-' '-' '-' '-' '-' '-' '-']
     ['-' '-' '-' '-' '-' '-' 'O']
     ['-' '-' '-' '-' '-' 'O' 'X']
     ['-' '-' '-' '-' 'O' 'X' 'X']
     ['-' '-' 'O' 'O' 'X' 'O' 'X']]
    -------------------------------
    [['1' '2' '3' '4' '5' '6' '7']]
    Player 2, Enter Postion (1-7): 
```

> The amount of stones to win can be changed by changing the parameter `terminating_length`


In [1]:
import numpy as np
import random
import json
import hashlib

## Game Configuration

To use a alredy trained Action-Value function set useOld to True

In [2]:
play = True
useOld = False
rows_count = 6
columns_count = 7
terminating_length = 4

### Players

In [3]:
ai = 0 
p = 1
players = ['O','X']

### Initialize AI Parameters

The high e value is needed to explore in the beginning as much as possible states, it can be lowered after around 1-2 Million episodes when there are enough states discovered

In [4]:
e = 0.4
alpha = 0.1
gamma = 1
columns = np.arange(columns_count)
actionQ = {}
episodes = 100000

### Rewards

In [5]:
rStep = 0
rDraw = -10
rLose = -100
rWin = 100
rInvalid = -2

### Play Configuration

changes the exploration rate to 0 and use trained data to play against a real player

In [6]:
if play:
    ## useOld = True ## Use not in Jupyter
    e = 0

### Use Trained AI

Load existing State-Action-Value function from Json

> Doesnt work in Jupyter Notebook

In [7]:
if(useOld):
    try:
        actionQ = json.load(open('actionQ.json'))
        print('old ActionQ loaded')
    except IOError:
        print('no old data')

### End Conditions

In [8]:
## Recrusive function to check if there is a row of stones
def checkPos(field, pos, player, mv, stw):
    if(stw < 1): 
        return True ## Stops when reaching the Terminating Length
    if (pos[0] < 0 or pos[0] >= rows_count) or (pos[1] < 0 or pos[1] >= columns_count):
        return False ## Stops when reaching the end of the Gamefield
    if(field[pos[0]][pos[1]] != player):
        return False ## Stops when finding a value that is not the player 
    return True and checkPos(field, (pos[0]+mv[0], pos[1]+mv[1]), player, mv, stw - 1) ## checks the next position


def winningConditions(field, player):
    for r in range(rows_count):
        for c in range(columns_count):
            if(field[r][c] == player):
                if(checkPos(field, (r, c), player, (1, 0), terminating_length)): # Horizontal
                    #print("H")
                    return True
                if(checkPos(field, (r, c), player, (0, 1), terminating_length)): # Vertical
                    #print("V")
                    return True
                if(checkPos(field, (r, c), player, (-1,1), terminating_length)): # Diagonal Left
                    #print("DL")
                    return True
                if(checkPos(field, (r, c), player, (1,1), terminating_length)): # Diagonal Right
                    #print("DR")
                    return True
    return False

def draw(field): ## Checks if there are an empty position in the Gamefield
    for r in range(rows_count):
        for c in range(columns_count):
            if field[r][c] == -1:
                return False
    return True

## Bot

Random player to train our AI

He has some logic to finish a game in vertical or horizontal when it is possible

In [9]:
def Trainer(field):
    col = -1
    ## Strategy to finish the game when it has a row of stones that is one smaller than the terminating length
    for r in range(rows_count):
        for c in range(columns_count):
            if(field[r][c] == p):
                if(checkPos(field, (r, c), p, (1, 0), terminating_length-1)): # Horizontal
                    if r-1 >= 0:
                        if field[r-1][c] == -1:
                            col = c
                            break
                if(checkPos(field, (r, c), p, (0, 1), terminating_length-1)): # Vertical
                    if c-1 >= 0:
                        if field[r][c-1] == -1:
                            col = c-1
                            break
                    if c+terminating_length-1 < columns_count:
                        if field[r][c+terminating_length-1] == -1:
                            col = c + terminating_length-1
                            break
    weights = np.full(columns_count, 1)
    if col >= 0 and col < columns_count:
        weights[col] = 7 ## Possibility to Finish the game when its possible 
    return random.choices(columns, weights)

## Game Funcions

Because the gamefield represents our state we used a hash of our field to save storagespace in our Action-Value function

> There are [4531985219092](https://oeis.org/A212693) possible states in the game ConnectFour with a 6x7 grid

In [10]:
def placeStone(field, column, player):
    for r in reversed(range(rows_count)):
        if(field[r][column] == -1):
            field[r][column] = player
            return True
    return False # row is full

## Displays the Gamefield for the Player
def printField(field):
    nField = np.chararray((rows_count, columns_count), itemsize=1, unicode=True)
    numbers = np.chararray((1, columns_count), itemsize=1, unicode=True)
    for r in range(rows_count):
        for c in range(columns_count):
            if(field[r][c] != -1):
                nField[r][c] = players[field[r][c]]
            else:
                nField[r][c] = '-'
    for c in range(columns_count):
        numbers[0][c] = str(c+1)
    print(nField)
    print('-' * (columns_count * 5) )
    print(numbers)

## Creates a Hash String for the current Gamefield
def hashField(field):
    return hashlib.sha1(field).hexdigest()

## Player 2

Player 2 is part if the environment in the perspective of our AI, it can be represented by the random Trainer or a real Player.



In [11]:
def opponentTurn(field, p, ai):
    
    ## Checks if AI Wins
    if (winningConditions(field, ai)):
        return rWin  # AI Winns
    if (draw(field)):
        return rDraw
    
    if not play: ## Random Player
        while True:
            if (placeStone(field, Trainer(field), p)):
                break
    else: ## Real Player
        printField(field)
        while True:
            try:
                column = int(input('Player ' + str(p+1) + ', Enter Postion (1-' + str(columns_count)+'): '))
                if (column > 0 and column <= columns_count):
                    column -= 1 ## Correct input for Array
                    if (placeStone(field, column, p)):
                        break
                    print('Invalid Input')
                print('Only Numbers between 1 and ' + str(columns_count))
            except ValueError:
                print('Only Numbers')
                continue


    ## Checks if Player Wins
    if (winningConditions(field, p)):
        return rLose  # Player Winns
    if (draw(field)):
        return rDraw
    return rStep

### e-Greedy

If there are several max values we chose randomly one of them

In [12]:
def policyDecideColumn(hState, actionQ):
    weights = np.full(len(columns), e / len(columns))
    ## Check if we know the state already
    if hState in actionQ:
        if play:
            print(actionQ[hState])
        result = np.argwhere(actionQ[hState] == np.amax(actionQ[hState])) ## get a Subset with the index of the lages values
        bestColumn = random.choice(result)[0] ## chooses a random value from the subset when there are multible maxValues
        weights[bestColumn] += 1 - e ## Set weight for the BestColumn to exploit the State
    col = random.choices(columns, weights) ## chooses the final column based on the weights (exploration, exploitation)
    return col[0]

### TD(0)

Why TD(0)
- don't need to discover hole state set
- can improve during episode

In [13]:
def playGame():
    field = np.full((rows_count, columns_count), -1)
    column = policyDecideColumn(hashField(field), actionQ)
    while True:
        oldField = np.copy(field) ## S
        oldColumn = column ## A
        if (placeStone(field, oldColumn, ai)):  ## Take Action
            reward = opponentTurn(field, p, ai) ## Reward
            hState = hashField(field) ## S'
            if not ( hState in actionQ): ## creates empty state if it not exists
                actionQ[hState] = np.zeros(columns_count).tolist()

        else:  ## Invalid Action
            reward = rInvalid ## Negative reward when it trys to choose a full row

        column = policyDecideColumn(hashField(field), actionQ) ## A'

        hOldState = hashField(oldField)
        hState = hashField(field)
        if hOldState in actionQ and hState in actionQ:
            ## Calculate new Value for the last State
            actionQ[hOldState][oldColumn] = actionQ[hOldState][oldColumn] + alpha * (
                    reward + gamma * actionQ[hState][column] - actionQ[hOldState][oldColumn])
        else:
            print("error")

        if reward > rStep or reward <= rDraw:
            # stops game when someone Wins
            return reward, field

### Initial Start State

When there are no values in the Action-Value Function we set the empty gamefield as a inital state

In [14]:
if len(actionQ) == 0:
    field = np.full((rows_count, columns_count), -1)
    actionQ[hashField(field)] = np.zeros(columns_count).tolist()

Debug Values

In [15]:
last = 0
drw = 0
win = 1
lose = 1

## Train AI / Play Game

In this part we can play the game against the trained AI or iterate through the episodes to train the AI

In [16]:
if play:
    while True:
        rwd, field = playGame()
        if rwd < rDraw:
            print('You Win')
        if rwd > rStep:
            print('You Loose')
        printField(field)
        if not (input('Restart? (y/n): ') == 'y'):
            break
else:
    for i in range(episodes):
        newReward, field = playGame()
        if newReward == -10:
            drw += 1
        if newReward == -100:
            lose += 1
        if newReward == 100:
            win += 1
        # reward += newReward1
        if i % 10000 == 0:
            print(len(actionQ)-last)
            print(str(i/episodes*100) + '%')
            print('Draw: ' + str(drw) + ' Win/Lose ' + str(win/lose))
            last = len(actionQ)
            # e -= 0.001
            #value = actionQ.values()
            #print (key, end=' => ')
            #print(value)
            #print(i)
        if i > 999999:
            play = True
            e = 0

print("Result:")
print(len(actionQ))
printField(field)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[['-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-']
 ['O' '-' '-' '-' '-' '-' '-']]
-----------------------------------
[['1' '2' '3' '4' '5' '6' '7']]
Player 2, Enter Postion (1-7): 7
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[['-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-']
 ['O' '-' '-' '-' '-' '-' '-']
 ['O' '-' '-' '-' '-' '-' 'X']]
-----------------------------------
[['1' '2' '3' '4' '5' '6' '7']]
Player 2, Enter Postion (1-7): 2
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[['-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-']
 ['O' '-' '-' '-' '-' '-' '-']
 ['O' 'X' '-' '-' 'O' '-' 'X']]
-----------------------------------
[['1' '2' '3' '4' '5' '6' '7']]
Player 2, Enter Postion (1-7): 3
[0.0, 0.0, 0.0, 0.0, 0.0, 0.

## Save ActionQ 

Saves the Action-Value function in a Json to reuse it in another session

In [17]:
json = json.dumps(actionQ)
f = open("ActionQ.json","w")
f.write(json)
f.close()